In [2]:
pip install python-dotenv 'crewai[tools]' langchain-openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [4]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
import os

In [5]:
llm = ChatOpenAI(model="gpt-5.2", temperature=0.5)

In [6]:
interviewer = input("Enter the name of the interviewer: ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

In [7]:
interviewer_agent = Agent(
    role=f"Você é {interviewer}, empregado na {company}, e está contratando para a vaga {job_position}",
    goal=f"Você ajuda o usuário a se preparar para a entrevista de emprego para a vaga {job_position} com a seguinte descrição: {job_description}",
    backstory="""Experience
    BEAT81
    Analytics Engineer

    BEAT81 · Full-timeBEAT81 · Full-time
    Apr 2024 - Present · 11 mos
    Berlin, Germany
    diconium
    Full-time · 4 yrs 2 mosFull-time · 4 yrs 2 mos

    Senior Cloud Architect
    Apr 2022 - Mar 2024 · 2 yrs
    Microsoft Azure, DBT and +4 skills

    Data Engineer
    Jan 2021 - Mar 2022 · 1 yr 3 mosJan 2021 to Mar 2022 · 1 yr 3 mos
    DBT, AWS and +4 skills

    Data Scientist
    Feb 2020 - Dec 2020 · 11 mos
    SQL and Python
    4flow logo

    Data Scientist
    4flow · Full-time4flow · Full-time
    Jun 2019 - Dec 2019 · 7 mosJun 2019 to Dec 2019 · 7 mos
    SQL and Python
    DCMN
    Full-time · 3 yrs 3 mos

    Data Scientist
    Apr 2017 - Apr 2019 · 2 yrs 1 moApr 2017 to Apr 2019 · 2 yrs 1 mo
    SQL and Python
    Data Analyst / Business Intelligence Manager

    Feb 2016 - Mar 2017 · 1 yr 2 mosFeb 2016 to Mar 2017 · 1 yr 2 mos
    SQL and Python
    iversity logo

    Data Analyst
    iversity · Full-timeiversity · Full-time
    Dec 2014 - Dec 2015 · 1 yr 1 moDec 2014 to Dec 2015 · 1 yr 1 mo
    SQL and R

    Education
    Technische Universität Berlin
    Master of Science - MS, Computational Neuroscience
    Sep 2009 - Aug 2013

    Universität Osnabrück
    Bachelor of Science - BS, Cognitive ScienceBachelor of Science - BS, Cognitive Science""",
    llm=llm,
)


In [8]:
interview_prep_task = Task(
    description=f"Entreviste o usuário para a vaga {job_description} com as seguintes descrições: {job_description}",
    expected_output=f"""
Pergunte apenas uma pergunta para o usuário que seja relevante a vaga {job_position}
""",
    agent=interviewer_agent,
    human_input=True,
)

# AI Coach

In [9]:
coach_agent = Agent(
    role="Coach de entrevista",
    goal=f"Eu ajudo o usuário a se preparar para a entrevista de emprego para a posição {job_position} avaliando a relevância da resposta.",
    backstory="Você é um expert em entrevista de empregos",
    llm=llm,
)

In [10]:
coaching_task = Task(
    description="Dê feedback para o usuário sobre sua resposta",
    expected_output="Dê feedback para o usuário sobre sua resposta com bullet points no que foi bom, ruim, e no que pode ser levado para o proximo nível",
    agent=coach_agent,
    context=[interview_prep_task],
)

# AI Crew

In [ ]:
crew = Crew(
    agents=[interviewer_agent, coach_agent],
    tasks=[interview_prep_task],
    verbose=True,
    process=Process.sequential,
)
result = crew.kickoff()